Challenge Statement
---------------------

The Hamiltonian H of a quantum system is the observable that measures its total energy. A surprising result in physics is that we can use this operator to calculate how a given quantum system evolves in time. An initial state $∣ψ⟩$ will, after a time $t$, evolve into $U(t)∣ψ⟩$, where
$$U(t) = exp(-iHt)$$
is a unitary operator. The symbol exp denotes the matrix exponential, which isn't always easy to calculate. However, we can build quantum circuits that approximately apply U(t). One method to do this is known as Trotterization. When the Hamiltonian is a sum
$$H = \sum_{i=1}^{k} H_i$$

of a number k of Hermitian operators $H_i$ that do not necessarily commute, we can approximate U via
$U(t) \approx \prod_{j=1}^{n} \prod_{i=1}^{k} \exp\left(-\frac{i H_i t}{n}\right)$.
Here, $n∈N^+$ is known as the Trotterization depth. The larger n is, the better the approximation of U that we get. As a quantum circuit, the Trotterization of U reads as in the figure below.
![circuit](./images/ASimpleTrotterization_1.png)
In this challenge, we will approximate the evolution of a one-dimensional spin-chain. A simplified version of a Hamiltonian that describes the interaction between two neighbouring spins is
$H = \alpha X \otimes X + \beta Z \otimes Z.$
Your task is to use a Trotterization approximation of depth n to simulate time evolution for a time t under this Hamiltonian. The parameters α and β are to be kept arbitrary. You may find the IsingXX and IsingZZ gates useful for this problem.

Challenge code
------------------

You must complete the 'trotterize' function to build the Trotterization of the Hamiltonian given above. You may not use PennyLane's built-in time evolution ('qml.ApproxTimeEvolution', 'qml.evolve') nor 'qml.QubitUnitary', but feel free to check your answer locally using these built-in PennyLane functions!

Input
----------

As input to this problem, you are given:

* alpha (float): The coefficient α of the X⊗X term in the Hamiltonian.
* beta (float): The coefficient β of the Z⊗Z term in the Hamiltonian.
* time (float): The period t over which the system evolves under the action of the Hamiltonian.
* depth (int): The Trotterization depth n as explained above.

Output
---------------
This code will output a list(float) (list of real numbers) corresponding to the probabilities of measuring ∣00⟩, ∣01⟩,∣10⟩, and ∣11⟩ (in that order) of the Trotterization circuit that you implement in PennyLane. The initial state of the circuit is ∣00⟩ and all measurements are performed in the computational basis.

In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device('default.qubit', wires = 2)

@qml.qnode(dev)
def trotterize(alpha, beta, time, depth):
    """This quantum circuit implements the Trotterization of a Hamiltonian given by a linear combination
    of tensor products of X and Z Pauli gates.

    Args:
        alpha (float): The coefficient of the XX term in the Hamiltonian, as in the statement of the problem.
        beta (float): The coefficient of the YY term in the Hamiltonian, as in the statement of the problem.
        time (float): Time interval during which the quantum state evolves under the interactions specified by the Hamiltonian.
        depth (int): The Trotterization depth.

    Returns:
        (numpy.array): The probabilities of measuring each computational basis state.
    """


    # Put your code here #
    # Return the probabilities


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    dev = qml.device("default.qubit", wires=2)
    ins = json.loads(test_case_input)
    output = list(trotterize(*ins).numpy())

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your circuit does not give the correct probabilities."

    tape = trotterize.qtape

    names = [op.name for op in tape.operations]

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit is using the built-in PennyLane Trotterization!"
    assert names.count('Evolve') == 0, "Your circuit is using the built-in PennyLane Trotterization!"
    assert names.count('QubitUnitary') == 0, "Can't use custom-built gates!"


# These are the public test cases
test_cases = [
    ('[0.5,0.8,0.2,1]', '[0.99003329, 0, 0, 0.00996671]'),
    ('[0.9,1.0,0.4,2]', '[0.87590286, 0, 0, 0.12409714]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")